In [1]:
import pandas as pd
df = pd.read_csv('binary.csv')
df

,admit,gre,gpa,rank
0,0,380,3.61,3
1,1,660,3.67,3
2,1,800,4.00,1
3,1,640,3.19,4
4,0,520,2.93,4
...,...,...,...,...
395,0,620,4.00,2
396,0,560,3.04,3
397,0,460,2.63,2
398,0,700,3.65,2


In [2]:
#make dummy variables for rank
data = pd.concat([df, pd.get_dummies(df['rank'], prefix='rank')], axis=1)
data = data.drop('rank', axis=1)
data

,admit,gre,gpa,rank_1,rank_2,rank_3,rank_4
0,0,380,3.61,0,0,1,0
1,1,660,3.67,0,0,1,0
2,1,800,4.00,1,0,0,0
3,1,640,3.19,0,0,0,1
4,0,520,2.93,0,0,0,1
...,...,...,...,...,...,...,...
395,0,620,4.00,0,1,0,0
396,0,560,3.04,0,0,1,0
397,0,460,2.63,0,1,0,0
398,0,700,3.65,0,1,0,0


In [3]:
# Standarize features
for field in ['gre','gpa']:
    mean, std = data[field].mean(), data[field].std()
    data.loc[:,field] = (data[field]-mean)/std
data

,admit,gre,gpa,rank_1,rank_2,rank_3,rank_4
0,0,-1.798011,0.578348,0,0,1,0
1,1,0.625884,0.736008,0,0,1,0
2,1,1.837832,1.603135,1,0,0,0
3,1,0.452749,-0.525269,0,0,0,1
4,0,-0.586063,-1.208461,0,0,0,1
...,...,...,...,...,...,...,...
395,0,0.279614,1.603135,0,1,0,0
396,0,-0.239793,-0.919418,0,0,1,0
397,0,-1.105469,-1.996759,0,1,0,0
398,0,0.972155,0.683454,0,1,0,0


In [4]:
import numpy as np
np.random.seed(42)
sample = np.random.choice(data.index, size=int( len(data)*0.9 ), replace=False)
data, test_data = data.iloc[sample], data.drop(sample)
data

,admit,gre,gpa,rank_1,rank_2,rank_3,rank_4
209,0,-0.066657,0.289305,0,1,0,0
280,0,0.625884,1.445476,0,1,0,0
33,1,1.837832,1.603135,0,0,1,0
210,0,1.318426,-0.131120,0,0,0,1
93,0,-0.066657,-1.208461,0,1,0,0
...,...,...,...,...,...,...,...
393,1,0.279614,0.946220,0,1,0,0
134,0,-0.239793,-1.155908,0,1,0,0
306,1,-0.412928,-0.577822,1,0,0,0
383,0,0.625884,1.603135,1,0,0,0


In [8]:
# Split into features and targets
features, targets = data.drop('admit', axis=1), data['admit']
features_test, targets_test = test_data.drop('admit', axis=1), test_data['admit']

In [21]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

def sigmoid_prime(x):
    return sigmoid(x) * (1-sigmoid(x))

np.random.seed(42)

n_records, n_features = features.shape #360,6
last_loss = None

# initialize weights
weights = np.random.normal(scale=1/n_features**.5, size=n_features)

# Neural Network Hyper permeters
epochs = 1000
learnrate = 0.5

for e in range(epochs):
    del_w = np.zeros(weights.shape)
    for x, y in zip(features.values, targets):
        output = sigmoid(np.dot(x,weights))
        
        error = y - output
        error_term = error * output * (1-output)
        
        del_w += error_term * x
        
        weights += learnrate * del_w / n_records
        
        if e % (epochs/10) == 0:
            out = sigmoid( np.dot(features, weights))
            loss = np.mean( (out - targets)**2 )
            
            if last_loss and last_loss < loss:
                print("Train loss: ", loss, " Warning - Loss Increasing")
            else:
                print("Train loss: ", loss)
                
            last_loss = loss

# calculate accuracy on test data
test_out = sigmoid(np.dot(features_test, weights))
predictions = test_out > 0.5
accuracy = np.mean(predictions == targets_test)
print("Predicition accuracy: {:.3f}".format(accuracy))

Train loss:  0.2641790041019977
Train loss:  0.26417472965566063
Train loss:  0.2641615079155366
Train loss:  0.2641464912820564
Train loss:  0.2641118978332227
Train loss:  0.2640819558759907
Train loss:  0.2640389477455856
Train loss:  0.2639992186913413
Train loss:  0.2639523544144303
Train loss:  0.26390337673866077
Train loss:  0.26385862095385526
Train loss:  0.2638136788575853
Train loss:  0.2637595003534591
Train loss:  0.263699523752723
Train loss:  0.263629770674233
Train loss:  0.2635674465165958
Train loss:  0.26351355923565273
Train loss:  0.26345568038568995
Train loss:  0.2633906904336899
Train loss:  0.26331492747685115
Train loss:  0.26323057195985733
Train loss:  0.2631401308535169
Train loss:  0.26304876738561583
Train loss:  0.26295043761983944
Train loss:  0.2628475037614048
Train loss:  0.26273316551882686
Train loss:  0.26262412702786925
Train loss:  0.2625162044997055
Train loss:  0.2624199980577825
Train loss:  0.2623223604336845
Train loss:  0.2622280752361219

Train loss:  0.21796458522781942  Warning - Loss Increasing
Train loss:  0.21795000906945172
Train loss:  0.2179400729654934
Train loss:  0.2179264901457714
Train loss:  0.21791360816121666
Train loss:  0.21790039900504393
Train loss:  0.21788725639160897
Train loss:  0.21787937752812755
Train loss:  0.21787165468776715
Train loss:  0.21786551938873064
Train loss:  0.21784564714300522
Train loss:  0.21782508021265856
Train loss:  0.2178047141877544
Train loss:  0.21778451238149638
Train loss:  0.21776382591583426
Train loss:  0.2177416214472148
Train loss:  0.2177191155941763
Train loss:  0.21769077222673186
Train loss:  0.2176624213565377
Train loss:  0.21763569263996113
Train loss:  0.21761368568724848
Train loss:  0.2175918998901247
Train loss:  0.21757508924382082
Train loss:  0.21755846263063472
Train loss:  0.21754203042676348
Train loss:  0.2175257851673229
Train loss:  0.21749471754928942
Train loss:  0.21747328484108588
Train loss:  0.2174494936420649
Train loss:  0.2174302109

Train loss:  0.21796458522239714
Train loss:  0.2179500090640299
Train loss:  0.21794007296007215
Train loss:  0.21792649014035728
Train loss:  0.21791360815581
Train loss:  0.21790039899964506
Train loss:  0.2178872563862179
Train loss:  0.21787937752274356
Train loss:  0.2178716546823902
Train loss:  0.2178655193833612
Train loss:  0.2178456471376433
Train loss:  0.2178250802073044
Train loss:  0.21780471418240765
Train loss:  0.217784512376157
Train loss:  0.2177638259105022
Train loss:  0.21774162144189022
Train loss:  0.21771911558885826
Train loss:  0.21769077222142094
Train loss:  0.21766242135123354
Train loss:  0.21763569263466395
Train loss:  0.21761368568195796
Train loss:  0.21759189988484143
Train loss:  0.21757508923854432
Train loss:  0.21755846262536555
Train loss:  0.21754203042150141
Train loss:  0.2175257851620683
Train loss:  0.21749471754404293
Train loss:  0.21747328483584671
Train loss:  0.21744949363683205
Train loss:  0.2174302109300442
Train loss:  0.217398829

Train loss:  0.21796458522239714
Train loss:  0.2179500090640299
Train loss:  0.21794007296007215
Train loss:  0.21792649014035734
Train loss:  0.21791360815581
Train loss:  0.2179003989996451
Train loss:  0.21788725638621795
Train loss:  0.21787937752274356
Train loss:  0.2178716546823902
Train loss:  0.21786551938336124
Train loss:  0.21784564713764334
Train loss:  0.21782508020730432
Train loss:  0.21780471418240765
Train loss:  0.21778451237615692
Train loss:  0.21776382591050225
Train loss:  0.21774162144189016
Train loss:  0.21771911558885826
Train loss:  0.21769077222142094
Train loss:  0.21766242135123354
Train loss:  0.21763569263466395
Train loss:  0.21761368568195796
Train loss:  0.21759189988484143
Train loss:  0.2175750892385444
Train loss:  0.21755846262536557
Train loss:  0.21754203042150147
Train loss:  0.2175257851620683
Train loss:  0.21749471754404293
Train loss:  0.21747328483584671
Train loss:  0.21744949363683203
Train loss:  0.2174302109300442
Train loss:  0.2173

Train loss:  0.21796458522239714
Train loss:  0.2179500090640299
Train loss:  0.21794007296007215
Train loss:  0.21792649014035734
Train loss:  0.21791360815581
Train loss:  0.2179003989996451
Train loss:  0.21788725638621795
Train loss:  0.21787937752274356
Train loss:  0.2178716546823902
Train loss:  0.21786551938336124
Train loss:  0.21784564713764334
Train loss:  0.21782508020730432
Train loss:  0.21780471418240765
Train loss:  0.21778451237615692
Train loss:  0.21776382591050225
Train loss:  0.21774162144189016
Train loss:  0.21771911558885826
Train loss:  0.21769077222142094
Train loss:  0.21766242135123354
Train loss:  0.21763569263466395
Train loss:  0.21761368568195796
Train loss:  0.21759189988484143
Train loss:  0.2175750892385444
Train loss:  0.21755846262536557
Train loss:  0.21754203042150147
Train loss:  0.2175257851620683
Train loss:  0.21749471754404293
Train loss:  0.21747328483584671
Train loss:  0.21744949363683203
Train loss:  0.2174302109300442
Train loss:  0.2173

...

Train loss:  0.21796458522239714
Train loss:  0.2179500090640299
Train loss:  0.21794007296007215
Train loss:  0.21792649014035734
Train loss:  0.21791360815581
Train loss:  0.2179003989996451
Train loss:  0.21788725638621795
Train loss:  0.21787937752274356
Train loss:  0.2178716546823902
Train loss:  0.21786551938336124
Train loss:  0.21784564713764334
Train loss:  0.21782508020730432
Train loss:  0.21780471418240765
Train loss:  0.21778451237615692
Train loss:  0.21776382591050225
Train loss:  0.21774162144189016
Train loss:  0.21771911558885826
Train loss:  0.21769077222142094
Train loss:  0.21766242135123354
Train loss:  0.21763569263466395
Train loss:  0.21761368568195796
Train loss:  0.21759189988484143
Train loss:  0.2175750892385444
Train loss:  0.21755846262536557
Train loss:  0.21754203042150147
Train loss:  0.2175257851620683
Train loss:  0.21749471754404293
Train loss:  0.21747328483584671
Train loss:  0.21744949363683203
Train loss:  0.2174302109300442
Train loss:  0.2173

Train loss:  0.21796458522239714
Train loss:  0.2179500090640299
Train loss:  0.21794007296007215
Train loss:  0.21792649014035734
Train loss:  0.21791360815581
Train loss:  0.2179003989996451
Train loss:  0.21788725638621795
Train loss:  0.21787937752274356
Train loss:  0.2178716546823902
Train loss:  0.21786551938336124
Train loss:  0.21784564713764334
Train loss:  0.21782508020730432
Train loss:  0.21780471418240765
Train loss:  0.21778451237615692
Train loss:  0.21776382591050225
Train loss:  0.21774162144189016
Train loss:  0.21771911558885826
Train loss:  0.21769077222142094
Train loss:  0.21766242135123354
Train loss:  0.21763569263466395
Train loss:  0.21761368568195796
Train loss:  0.21759189988484143
Train loss:  0.2175750892385444
Train loss:  0.21755846262536557
Train loss:  0.21754203042150147
Train loss:  0.2175257851620683
Train loss:  0.21749471754404293
Train loss:  0.21747328483584671
Train loss:  0.21744949363683203
Train loss:  0.2174302109300442
Train loss:  0.2173

Train loss:  0.21796458522239714
Train loss:  0.2179500090640299
Train loss:  0.21794007296007215
Train loss:  0.21792649014035734
Train loss:  0.21791360815581
Train loss:  0.2179003989996451
Train loss:  0.21788725638621795
Train loss:  0.21787937752274356
Train loss:  0.2178716546823902
Train loss:  0.21786551938336124
Train loss:  0.21784564713764334
Train loss:  0.21782508020730432
Train loss:  0.21780471418240765
Train loss:  0.21778451237615692
Train loss:  0.21776382591050225
Train loss:  0.21774162144189016
Train loss:  0.21771911558885826
Train loss:  0.21769077222142094
Train loss:  0.21766242135123354
Train loss:  0.21763569263466395
Train loss:  0.21761368568195796
Train loss:  0.21759189988484143
Train loss:  0.2175750892385444
Train loss:  0.21755846262536557
Train loss:  0.21754203042150147
Train loss:  0.2175257851620683
Train loss:  0.21749471754404293
Train loss:  0.21747328483584671
Train loss:  0.21744949363683203
Train loss:  0.2174302109300442
Train loss:  0.2173

Train loss:  0.21796458522239714
Train loss:  0.2179500090640299
Train loss:  0.21794007296007215
Train loss:  0.21792649014035734
Train loss:  0.21791360815581
Train loss:  0.2179003989996451
Train loss:  0.21788725638621795
Train loss:  0.21787937752274356
Train loss:  0.2178716546823902
Train loss:  0.21786551938336124
Train loss:  0.21784564713764334
Train loss:  0.21782508020730432
Train loss:  0.21780471418240765
Train loss:  0.21778451237615692
Train loss:  0.21776382591050225
Train loss:  0.21774162144189016
Train loss:  0.21771911558885826
Train loss:  0.21769077222142094
Train loss:  0.21766242135123354
Train loss:  0.21763569263466395
Train loss:  0.21761368568195796
Train loss:  0.21759189988484143
Train loss:  0.2175750892385444
Train loss:  0.21755846262536557
Train loss:  0.21754203042150147
Train loss:  0.2175257851620683
Train loss:  0.21749471754404293
Train loss:  0.21747328483584671
Train loss:  0.21744949363683203
Train loss:  0.2174302109300442
Train loss:  0.2173

Predicition accuracy: 0.725


In [44]:
1>0

True

In [28]:
out

array([0.24119422, 0.1532274 , 0.05763573, 0.10564512, 0.34694017,
       0.14087972, 0.21995636, 0.22115312, 0.15836779, 0.5309982 ,
       0.14493056, 0.57306524, 0.17311452, 0.16264118, 0.25806822,
       0.29473878, 0.15388083, 0.20640595, 0.15081121, 0.72476217,
       0.05763573, 0.16491463, 0.16037466, 0.16135963, 0.17268866,
       0.19332504, 0.07704496, 0.10185169, 0.18538634, 0.15206189,
       0.1570777 , 0.16654364, 0.36848651, 0.27628718, 0.34959797,
       0.308094  , 0.60168814, 0.10110173, 0.21349022, 0.09843159,
       0.25851688, 0.06482224, 0.15227091, 0.4010278 , 0.43354732,
       0.25507737, 0.71953731, 0.10698014, 0.20174848, 0.17624599,
       0.24785237, 0.27529755, 0.63545409, 0.07867513, 0.20477214,
       0.26488893, 0.08975205, 0.11151541, 0.10761895, 0.15871923,
       0.07452066, 0.78015393, 0.13299068, 0.65198485, 0.12478784,
       0.1066817 , 0.176193  , 0.2110344 , 0.11145241, 0.16844428,
       0.21247754, 0.15992615, 0.25519824, 0.16946881, 0.27409